# Projet_09 : Réalisez un traitement dans un environnement Big Data sur le Cloud

# Etape 2 : Migrez votre chaîne de traitement dans le cloud
---

## Import des librairies

In [2]:
# start spark session

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Une session Spark est automatiquement démarrée.

In [1]:
import pandas as pd
from PIL
import numpy as np
import io
import os

import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split, udf

from pyspark.ml.feature import PCA
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler

################################################################################
## Libraries version :

print(f"Pandas : {pd.__version__}")
print(f"Numpy : {np.__version__}")
print(f"Pillow : {PIL.__version__}")
print(f"Tensorflow : {tf.__version__}")
print(f"PySpark : {pyspark.__version__}")

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1738920115233_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

---
## Définition des emplacements :

Les images ont été chargées sur le cloud, dans mon bucket S3.  
Je définis les différents emplacements dans le bucket :
* Images de Test dans : *s3://jlegal-bucket/Test*
* Fichiers finaux en `parquet` : *s3://jlegal-bucket/Results*

In [3]:
PATH = 's3://jlegal-bucket'
PATH_Data = PATH+'/Test'
PATH_Result = PATH+'/Results'
print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Result: '+PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PATH:        s3://jlegal-bucket
PATH_Data:   s3://jlegal-bucket/Test
PATH_Result: s3://jlegal-bucket/Results

---
---
## 1.2 - Chargement des données

Les images sont chargées au format binaire, ce qui offre, <br />
plus de souplesse dans la façon de prétraiter les images.

Avant de charger les images, nous spécifions que nous voulons charger <br />
uniquement les fichiers dont l'extension est **jpg**.

Nous indiquons également de charger tous les objets possibles contenus <br />
dans les sous-dossiers du dossier communiqué.

In [4]:
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<u>Affichage des 5 premières images contenant</u> :
 - le path de l'image
 - la date et heure de sa dernière modification
 - sa longueur
 - son contenu encodé en valeur hexadécimal

<u>Je ne conserve que le **path** de l'image et j'ajoute une colonne contenant les **labels** de chaque image</u> :

In [5]:
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
print(images.printSchema())
print(images.select('path','label').show(5,False))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+----------------------------------------------------+--------------+
|path                                                |label         |
+----------------------------------------------------+--------------+
|s3://jlegal-bucket/Test/Apple Braeburn/r_326_100.jpg|Apple Braeburn|
|s3://jlegal-bucket/Test/Apple Braeburn/r_4_100.jpg  |Apple Braeburn|
|s3://jlegal-bucket/Test/Apple Braeburn/r_8_100.jpg  |Apple Braeburn|
|s3://jlegal-bucket/Test/Apple Braeburn/r_324_100.jpg|Apple Braeburn|
|s3://jlegal-bucket/Test/Apple Braeburn/r_327_100.jpg|Apple Braeburn|
+----------------------------------------------------+--------------+
only showing top 5 rows

None

---
---
## 1.3 - Préparation du modèle :

### 1.3.1 - Chargement du modèle : **MobileNetV2**

In [6]:
# load MobileNetV2 model 
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14536120/14536120 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step

---
### 1.3.2 - Création de notre modèle sans la dernière couche :

In [7]:
# create our model without last layer
new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Affichage du résumé de notre nouveau modèle où nous constatons <br />
que <u>nous récupérons bien en sortie un vecteur de dimension (1, 1, 1280)</u> :

In [8]:
new_model.summary()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model: "functional_1"
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │       0 │ -                    │
│ (InputLayer)        │ 3)                │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │     864 │ input_layer[0][0]    │
│                     │ 32)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │     128 │ Conv1[0][0]          │
│ (BatchNormalizatio… │ 32)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │       0 │ 

Tous les workeurs doivent pouvoir accéder au modèle ainsi qu'à ses poids. <br />
Une bonne pratique consiste à charger le modèle sur le driver puis à diffuser <br />
ensuite les poids aux différents workeurs.

In [9]:
brodcast_weights = sc.broadcast(new_model.get_weights())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<u>Mettons cela sous forme de fonction</u> :

In [10]:
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed 
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

---
---
## 1.4 - Création du pipeline :

Afin de palier à une très forte influence sur le site, j'ai activé **l'auto-scaling** dans mon EMR, je vérifie donc que `Spark` est configuré comme il faut :

In [ ]:
print(spark.conf.get("spark.dynamicAllocation.enabled"))

### 1.4.1 - Fonctions :

Je garde les fonctions utilisées en local, elles sont déjà optimisées avec le fait de cahrger une seule fois le modèle et de pouvoir traiter des `series` complètes.

In [11]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

@pandas_udf('array<float>'
def featurize_udf(content_series_iter: Iterator[pd.Series]) -> Iterator[pd.Series]:
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/mnt/yarn/usercache/livy/appcache/application_1738920115233_0001/container_1738920115233_0001_01_000001/pyspark.zip/pyspark/sql/pandas/functions.py:407: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.

Utilisation de `VectorAssembler` pour la production, plus efficace en pipeline.

---
### 1.4.2 - Vecteur dense :

Préparation de la vectorisation, à intégrer dans le pipeline.

In [24]:
# array to dense vector
to_vector_udf = udf(lambda x: Vectors.dense(x), VectorUDT())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
# convert "features" array in vector
vector_assembler = VectorAssembler(inputCols=["features_vec"], outputCol="features_vec_final")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

---
### 1.4.3 - PCA :

In [28]:
# PCA
k_optimal = 186
pca = PCA(k=k_optimal, inputCol="features_vec_final", outputCol="pca_features")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

---
### 1.4.4 - Pipeline :

In [29]:
# pipeline
pipeline = Pipeline(stages=[vector_assembler, pca])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 1.4.5 - Application du pipeline complet :

Application de la featurisation et de la vectorisation en une seule fois.

In [ ]:
# features and vectors
features_df = images.withColumn("features", featurize_udf(col("content"))) \
                    .withColumn("features_vec", to_vector_udf(col("features")))

In [ ]:
# fit 
pipeline_model = pipeline.fit(features_df)

In [ ]:
# transform
df_pca = pipeline_model.transform(features_df)

In [ ]:
df_pca.printSchema()

In [ ]:
df_pca.show(5)

Ecriture des résultats en fichier `parquet`.  
Optimisation avec `partitionBy` de Spark : [https://sparkbyexamples.com/pyspark/pyspark-partitionby-example/](https://sparkbyexamples.com/pyspark/pyspark-partitionby-example/)

In [ ]:
df_pca.write.partitionBy("label").mode("overwrite").parquet(PATH_Result)

Possibilité de sauvegarder le pipeline et de le recharger ensuite :

In [35]:
# save pipeline
pipeline_model.save(PATH +"/model/model_pca")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Recharge avec :

In [ ]:
from pyspark.ml import PipelineModel
pipeline_model_loaded = PipelineModel.load(PATH + "/model/model_pca")